In [341]:
#Importamos las librerías
import cv2
import os
import numpy as np 
import keras
import matplotlib.pyplot as plt
import glob

### Importación de librerías
En esta celda importamos las librerías necesarias para el procesamiento de imágenes, manejo de archivos, operaciones numéricas, construcción de modelos de deep learning y visualización de datos.

### Descompresión del dataset
Aquí sería donde descomprimo el dataset, pero como estoy usando jupyter, debo borrar la carpeta (si existe) y volver a descomprimirla. Luego veremos por qué es necesario borrar la carpeta cada vez que ejecutemos el cuaderno.

In [342]:
import shutil
import zipfile
import os

def eliminar_carpetas(carpeta):
    if os.path.exists(carpeta):
        shutil.rmtree(carpeta)
        print(f"Carpeta '{carpeta}' eliminada.")
    else:
        print(f"La carpeta '{carpeta}' no existe.")

def extraer_zip(archivo_zip, destino):
    with zipfile.ZipFile(archivo_zip, 'r') as zip_ref:
        zip_ref.extractall(destino)
        print(f"Archivo '{archivo_zip}' extraído en '{destino}'.")

# Ejemplo de uso
carpeta_a_eliminar = 'ruta/a/tu/carpeta'
archivo_zip = 'ruta/a/tu/archivo.zip'
destino_extraccion = 'ruta/a/tu/destino'

eliminar_carpetas("Chessman-image-dataset")
extraer_zip("archive.zip", ".")


Carpeta 'Chessman-image-dataset' eliminada.
Archivo 'archive.zip' extraído en '.'.


### Mapeo de clases y tamaño de imágenes
En esta celda definimos un diccionario que mapea los nombres de las piezas de ajedrez a etiquetas numéricas y establecemos el tamaño estándar de las imágenes a 150x150 píxeles.

In [343]:
#Hacemos una relación entre los nombres de las imágenes y las clases
MAP_CHARACTERS = { 0: 'Bishop', 1: 'King', 2: 'Knight', 3: 'Pawn', 4: 'Queen', 5: 'Rook' }
# Vamos a standarizar todas las imágenes a tamaño 64x64
IMG_SIZE = 150

### Funciones para cargar los datos
Declaramos dos funciones para cargar los datos. Una dedicada a la fracción train y la otra para test.

In [344]:
def load_train_set(dirname, map_characters, verbose=True):
	"""Esta función carga los datos de training en imágenes.

	Como las imágenes tienen tamaños distintas, utilizamos la librería opencv
	para hacer un resize y adaptarlas todas a tamaño IMG_SIZE x IMG_SIZE.

	Args:
		dirname: directorio completo del que leer los datos
		map_characters: variable de mapeo entre labels y personajes
		verbose: si es True, muestra información de las imágenes cargadas

	Returns:
		X, y: X es un array con todas las imágenes cargadas con tamaño
	IMG SIZE x IMG SIZE
				y es un array con las labels de correspondientes a cada imagen
	"""
	X_train = []
	y_train = []
	for label, character in map_characters.items():        
		files = os.listdir(os.path.join(dirname, character))
		images = [file for file in files if file.endswith("jpg")]
		if verbose:
			print("Leyendo {} imágenes encontradas de {}".format(len(images), character))
		for image_name in images:
			image = cv2.imread(os.path.join(dirname, character, image_name))
			X_train.append(cv2.resize(image,(IMG_SIZE, IMG_SIZE)))
			y_train.append(label)
	return np.array(X_train), np.array(y_train)

def load_test_set(dirname, map_characters, verbose=True):
   """Esta función funciona de manera equivalente a la función load_train_set
   pero cargando los datos de test."""
   X_test = []
   y_test = []
   reverse_dict = {v: k for k, v in map_characters.items()}
   for filename in glob.glob(dirname + '/*.*'):
       char_name = "_".join(filename.split('/')[-1].split('_')[:-1])
       if char_name in reverse_dict:
           image = cv2.imread(filename)
           image = cv2.resize(image, (IMG_SIZE, IMG_SIZE))
           X_test.append(image)
           y_test.append(reverse_dict[char_name])
   if verbose:
       print("Leídas {} imágenes de test".format(len(X_test)))
   return np.array(X_test), np.array(y_test)
   

### División del dataset en train y test
Uso esta función para dividir el dataset en train y test.

In [345]:
import shutil
from sklearn.model_selection import train_test_split


def split_dataset(source_dir, dest_dir, split_ratio=0.3):
	"""
	Esta función divide el dataset en dos partes, una para training y otra para test, y lo pone en directorios distintos.
	"""
	if not os.path.exists(dest_dir):
		os.makedirs(dest_dir)
	
	for character in os.listdir(source_dir):
		character_path = os.path.join(source_dir, character)
		if os.path.isdir(character_path):
			images = [file for file in os.listdir(character_path) if file.endswith("jpg")]
			train_images, test_images = train_test_split(images, test_size=split_ratio, random_state=42)
			
			character_dest_path = os.path.join(dest_dir, character)
			if not os.path.exists(character_dest_path):
				os.makedirs(character_dest_path)
			
			for image in test_images:
				shutil.move(os.path.join(character_path, image), os.path.join(character_dest_path, image))

source_directory = ".\\Chessman-image-dataset\\Chess"
destination_directory = ".\\Chessman-image-dataset\\Chess_test"
split_dataset(source_directory, destination_directory)

### Aumento de datos
En esta celda utilizamos `ImageDataGenerator` de Keras para realizar aumentos de datos, como rotaciones, desplazamientos, zoom, etc., para mejorar la generalización del modelo.

In [346]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Aumento de datos
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)


### Carga de datos
Cargamos los datos de entrenamiento y prueba desde los directorios especificados y normalizamos las imágenes dividiendo los valores de los píxeles por 255.0.

In [347]:
# Cargamos los datos. Si no estás trabajando en colab, cambia los paths por
# los de los ficheros donde hayas descargado los datos.
DATASET_TRAIN_PATH_COLAB = ".\\Chessman-image-dataset\\Chess"
DATASET_TEST_PATH_COLAB = "Chessman-image-dataset\\Chess_test"
X, y = load_train_set(DATASET_TRAIN_PATH_COLAB, MAP_CHARACTERS)
X_t, y_t = load_test_set(DATASET_TEST_PATH_COLAB, MAP_CHARACTERS)
X = X / 255.0
X_t = X_t / 255.0
# Vamos a barajar aleatoriamente los datos. Esto es importante ya que si no
# lo hacemos y, por ejemplo, cogemos el 20% de los datos finales como validation
# set, estaremos utilizando solo un pequeño número de personajes, ya que
# las imágenes se leen secuencialmente personaje a personaje.
perm = np.random.permutation(len(X))
X, y = X[perm], y[perm]

Leyendo 49 imágenes encontradas de Bishop
Leyendo 42 imágenes encontradas de King
Leyendo 65 imágenes encontradas de Knight
Leyendo 60 imágenes encontradas de Pawn
Leyendo 46 imágenes encontradas de Queen
Leyendo 60 imágenes encontradas de Rook
Leídas 0 imágenes de test


### División de datos en entrenamiento y validación
Separamos los datos en conjuntos de entrenamiento y validación utilizando `train_test_split` de `sklearn`.

In [348]:
from sklearn.model_selection import train_test_split

# Separamos los datos en training y validation
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Training data shape: {X_train.shape}")
print(f"Validation data shape: {X_val.shape}")

Training data shape: (257, 150, 150, 3)
Validation data shape: (65, 150, 150, 3)


### Visualización de ejemplos
Declaramos una función para visualizar las imágenes y mostramos un ejemplo de imagen normalizada.

In [349]:
 #Declaramos una función para visualizar las imágenes
def visualize_example(x):
	plt.figure()
	plt.imshow(x)
	plt.colorbar()
	plt.grid(False)
	plt.show()
print("Número de elementos en X: ",len(X))
# for i in range(len(X)):
# 	visualize_example(X[i])
# 	print(MAP_CHARACTERS[y[i]])
 
visualize_example(X[70]) # Visualiza la imagen normalizada con valores de 0 a 1
print(MAP_CHARACTERS[y[70]]) # Acceso al diccionario
print(X[70].shape[0:3]) # Dimensiones de la imagen tras resize con los 3 canales RGB

Número de elementos en X:  322
